In [1]:
'''
EDA Starter pack
'''

'\nEDA Starter pack\n'

In [2]:
import numpy as np
import pandas as pd

from scipy.stats import pearsonr

# Visualiazation
import seaborn as sns
import matplotlib.pyplot as plt

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

path_data = 'input_files/ASHRAE/'
path_train = path_data + 'train.csv'
path_test = path_data + 'test.csv'

In [3]:
def column_visualizatin(df_, col_, target_, koef_, with_target_=False):

    fig, ax = plt.subplots()
    ax.hist(df_[col_], color='g', alpha=0.5, normed=True, label=col_)

    if with_target_:
        df_grouped_ = df_[[col_, target_]].groupby([col_]).mean().reset_index()
        df_grouped_[target_] = df_grouped_[target_].astype('float') / koef_
        ax.plot(df_grouped_[col_],
                df_grouped_[target_],
                color='r',
                label=target_)

    ax.set(title=col_)
    ax.legend(loc='best')
    plt.show()


def share_of_missing_per_column(df_):
    ''' Share of missing values per column '''

    data = []
    for col in df_.columns:
        if (df_[col].isnull().sum() > 0):
            data.append([
                col, '{}%'.format(
                    round(100 * df_[col].isnull().sum() / df_[col].shape[0],
                          2)), df_[col].isnull().sum()
            ])

    if (len(data) == 0):
        print('No missing data')
    else:
        return pd.DataFrame.from_records(
            data, columns=['Column', 'Missing_share',
                           'Missing_amount']).sort_values(by=['Missing_share'],
                                                          ascending=False)

## Import
read Train/Test sets 

In [4]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)
target = 'meter_reading'

In [5]:
target = 'meter_reading'

## Overview
simple information

In [6]:
print('Train shape:', df_train.shape)
df_train.head(2)

Train shape: (20216100, 4)


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0


In [7]:
print('Test shape:', df_test.shape)
df_test.head(2)

Test shape: (41697600, 4)


,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00


## Target
info abou target variable

In [8]:
# Check the share
df_train[target].value_counts(normalize=True)

0.0000       9.269721e-02
20.0000      1.155663e-03
2.9307       1.146660e-03
36.6000      1.095859e-03
8.7921       1.077705e-03
24.4000      1.069049e-03
5.8614       1.054061e-03
30.0000      1.031505e-03
61.0000      1.030169e-03
10.0000      1.024530e-03
33.5500      9.902503e-04
25.0000      9.566138e-04
4.0000       9.489466e-04
24.0000      9.334639e-04
21.3500      9.288142e-04
27.4500      9.273797e-04
18.0000      8.712858e-04
30.5000      8.386880e-04
100.0000     8.142025e-04
6.0000       8.126691e-04
31.0000      7.854136e-04
27.0000      7.731462e-04
26.0000      7.715633e-04
21.0000      7.688427e-04
36.0000      7.671608e-04
39.6500      7.605324e-04
28.0000      7.594442e-04
2.0000       7.591969e-04
11.7228      7.487102e-04
29.0000      7.414882e-04
                 ...     
3003.8000    4.946553e-08
2976.9500    4.946553e-08
57.2411      4.946553e-08
436.1820     4.946553e-08
24.4296      4.946553e-08
91.6348      4.946553e-08
28.2318      4.946553e-08
60.0158     

In [9]:
# sns.countplot(x=target, data=df_train)
# plt.xticks(rotation=90)

In [10]:
# df_train[target].plot()
# plt.show()

## Missing data
share of missing values per column

In [11]:
share_of_missing_per_column(df_train)

No missing data


In [12]:
share_of_missing_per_column(df_test)

No missing data


## Data types
cstegorical/numerical columns identification

In [13]:
cat_cols, num_cols = [], []

for col in df_train.columns:
    if df_train[col].dtype == object:
        cat_cols.append(col)
    else:
        num_cols.append(col)
print('Numerical columns {}, categorical columns {}'.format(
    len(num_cols), len(cat_cols)))
cat_cols

Numerical columns 3, categorical columns 1


['timestamp']

In [14]:
df_train.dtypes.value_counts()

int64      2
object     1
float64    1
dtype: int64

In [15]:
df_test.dtypes.value_counts()

int64     3
object    1
dtype: int64

## Staistic

In [16]:
df_train.describe()

,building_id,meter,meter_reading
count,2.021610e+07,2.021610e+07,2.021610e+07
mean,7.992780e+02,6.624412e-01,2.117121e+03
std,4.269133e+02,9.309921e-01,1.532356e+05
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.930000e+02,0.000000e+00,1.830000e+01
50%,8.950000e+02,0.000000e+00,7.877500e+01
75%,1.179000e+03,1.000000e+00,2.679840e+02
max,1.448000e+03,3.000000e+00,2.190470e+07


In [17]:
df_test.describe()

,row_id,building_id,meter
count,4.169760e+07,4.169760e+07,4.169760e+07
mean,2.084880e+07,8.075824e+02,6.642857e-01
std,1.203706e+07,4.297680e+02,9.278067e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.042440e+07,4.047500e+02,0.000000e+00
50%,2.084880e+07,9.000000e+02,0.000000e+00
75%,3.127320e+07,1.194250e+03,1.000000e+00
max,4.169760e+07,1.448000e+03,3.000000e+00


## Train VS Test 
numerical statistic per column

In [19]:
data, lst = [], df_train.columns
lst.pop(lst.index(target))

for col in lst:
    if col in num_cols:
        eql_median = False

        if df_train[col].median() == df_test[col].median():
            eql_median = True

        data.append([
            col, df_train[col].median(), df_test[col].median(), eql_median,
            df_train[col].mean(), df_test[col].mean(), df_train[col].var(),
            df_test[col].var()
        ])
pd.DataFrame.from_records(data,
                          columns=[
                              'Column',
                              'Train_median',
                              'Test_median',
                              'Equal_medians',
                              'Train_mean',
                              'Test_mean',
                              'Train_var',
                              'Test_var',
                          ]).head(5)

AttributeError: 'Index' object has no attribute 'pop'

## Correlation

In [ ]:
cmap = sns.diverging_palette(h_neg=10, h_pos=240, as_cmap=True)

# Create the correlation matrix
corr = df_train.corr()

# Draw the heatmap
sns.heatmap(corr, cmap = cmap, center=0, linewidths=1, annot=True, fmt=".2f")
plt.show()

In [ ]:
# Generate a mask for the upper triangle 
mask = np.triu(np.ones_like(corr, dtype=bool))

# Add the mask to the heatmap
sns.heatmap(corr, cmap = cmap, mask=mask, center=0, linewidths=1, annot=True, fmt=".2f")
plt.show()

In [ ]:
# the strongest correlation is between: meter and building_id

In [ ]:
# calculate Pearson's correlation
# corr, _ = pearsonr(df_train['65'], df_train['66'])

In [ ]:
# plt.figure(figsize=(8, 8))
# sns.heatmap(df_train.corr(), square=True, annot=True)

In [ ]:
abs(df_train.corr().unstack()).sort_values(
    ascending=False).drop_duplicates()[1:10]

## Columns

In [ ]:
# column_visualizatin(df_train, df_train.columns[0], target, 100, True)

In [ ]:
# column_visualizatin(df_train, df_train.columns[1], target, 100, True)

In [ ]:
# column_visualizatin(df_train, df_train.columns[2], target, 100, True)

In [ ]:
# column_visualizatin(df_train, df_train.columns[3], target, 100, True)